# Agent Evaluation with RAGAS

We will use the RAGAS library to evaluate the performance our Gemini agent. RAGAS is a framework designed to assess the capabilities of AI agents in various tasks.

This notebook uses the functions in `ragas_eval.py` to evaluate our RAG pipeline
across different hyperparameter combinations.

The parameters for the evaluation are as follows:
* chunk_size
* top_k
* temperature

In [1]:
%load_ext autoreload
%autoreload 2

## Mount on Colab

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
ROOT_DIR = "/content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/model_evaluation"
%cd $ROOT_DIR

/content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/model_evaluation


In [ ]:
%ls $ROOT_DIR

Evaluate_Model.ipynb  ragas_eval.py


In [4]:
!pip install langchain-community langchain-google-genai duckduckgo-search pypdf faiss-cpu python-dotenv ragas pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9

In [2]:
# Imports
import sys
sys.path.append('..')

import os
import sys
import warnings
import pandas as pd
from datetime import datetime

# Suprimir warnings innecesarios
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from generative_resp import pdf_process_utils
import ragas_eval as ragas_eval

project_root = os.path.dirname(ROOT_DIR)
sys.path.append(project_root)

In [3]:
def main():
    """
    Función principal que carga los documentos y ejecuta la evaluación.
    """
    print("\n" + "="*60)
    print("🚀 INICIANDO EVALUACIÓN RAG SIMPLIFICADA")
    print(f"📅 Fecha y Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)

    # Definir la ruta al directorio de los PDFs
    pdf_directory = os.path.join(project_root, 'generative_resp', 'polizas')

    if not os.path.isdir(pdf_directory):
        print(f"❌ Error: El directorio de PDFs no se encuentra en: {pdf_directory}")
        return

    print(f"📁 Cargando documentos desde: {pdf_directory}...")

    # Cargar los documentos. La división inicial no es crítica, ya que cada
    # configuración la re-dividirá según su 'chunk_size'.
    all_documents = pdf_process_utils.load_split_pdfs(
        pdf_dir=pdf_directory,
        chunk_size=1000,
        chunk_overlap=100
    )

    if not all_documents:
        print("❌ No se pudieron cargar los documentos. Abortando.")
        return

    print(f"✅ {len(all_documents)} chunks de documentos cargados inicialmente.")

    # --- Ejecutar la evaluación ---
    print("\n▶️ Lanzando el proceso de evaluación de RAGAS...")

    try:
        results_df = ragas_eval.run_evaluation(original_docs=all_documents)

        if results_df is not None:
            print("\n" + "="*60)
            print("🎉 ¡EVALUACIÓN COMPLETADA!")
            print("="*60)
            print("📊 Resumen de Resultados Finales (Promedio por combinación):")

            # Imprimir el DataFrame de resultados con un formato más limpio
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 100)
            pd.set_option('display.colheader_justify', 'center')
            pd.set_option('display.precision', 4)

            print(results_df)

            # Guardar resultados en un archivo CSV
            results_path = 'ragas_simplified_results.csv'
            results_df.to_csv(results_path, index=False)
            print(f"\n💾 Resultados guardados en: {results_path}")

        else:
            print("\n❌ La evaluación no produjo resultados.")

    except Exception as e:
        print(f"\n❌ Ocurrió un error crítico durante la evaluación: {e}")
        import traceback
        traceback.print_exc()

In [4]:
if __name__ == "__main__":
    main()


🚀 INICIANDO EVALUACIÓN RAG SIMPLIFICADA
📅 Fecha y Hora: 2025-06-16 00:02:29
📁 Cargando documentos desde: /content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/generative_resp/polizas...
Split 267 documents into 699 chunks.
✅ 699 chunks de documentos cargados inicialmente.

▶️ Lanzando el proceso de evaluación de RAGAS...
Configurando dependencias de RAGAS (LLM y Embeddings)...


/content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/model_evaluation/ragas_eval.py:54: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

    ✅ Éxito (resultado procesado desde lista).
    ⏳ Pausa de 180 segundos para proteger la cuota...
  - Evaluando pregunta 3/4...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

    ✅ Éxito (resultado procesado desde lista).
    ⏳ Pausa de 180 segundos para proteger la cuota...
  - Evaluando pregunta 4/4...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

    ✅ Éxito (resultado procesado desde lista).
    ⏳ Pausa de 180 segundos para proteger la cuota...

  📊 Resultados promedio para la configuración:
     - faithfulness: 1.0000
     - answer_relevancy: 0.6481
     - context_precision: 0.3750
     - context_recall: 0.5000

🎉 ¡EVALUACIÓN COMPLETADA!
📊 Resumen de Resultados Finales (Promedio por combinación):
    combination_name    faithfulness  answer_relevancy  context_precision  context_recall
0  Creative & Balanced      0.95           0.6379             0.375              0.5     
1   Wide Context Sweep      1.00           0.4784             0.375              0.5     
2     Balanced Natural      1.00           0.6302             0.375              0.5     
3      Dense Precision      1.00           0.6481             0.375              0.5     

💾 Resultados guardados en: ragas_simplified_results.csv


## Results

After running the evaluation, we will obtain a table with the results for each combination of hyperparameters.

| **Name of combination** | **chunk_size** | **top_k** | **temperature** |
|------------------|------------------|----------|----------|
| Creative & Balanced          | 1000          | 4     | 0.3     |
| Wide Context Sweep          | 1200          | 6     | 0.1     |
| Balanced Natural          | 1000          | 3     | 0.2     |
| Dense Precision          | 1800          | 2     | 0.15     |

 The results were saved on **ragas_simplified_results.csv**


In [5]:
results = pd.read_csv('ragas_simplified_results.csv')
results

,combination_name,faithfulness,answer_relevancy,context_precision,context_recall
0,Creative & Balanced,0.95,0.6379,0.375,0.5
1,Wide Context Sweep,1.00,0.4784,0.375,0.5
2,Balanced Natural,1.00,0.6302,0.375,0.5
3,Dense Precision,1.00,0.6481,0.375,0.5


The best hyperparameters combination is:
**Dense Precision**:

* chunk_size: *1800*
* top_k: *2*  
* temperature: *0.15*